In [246]:
import re

import requests
from bs4 import BeautifulSoup

#part 1: https://pumpout.anyhowstep.com/ 으로부터의 데이터 추출: metadata확보
#part 2: piugame.com의 선곡 랭킹으로부터의 데이터 추출: 작곡가/노래 이름 한-영 번역 기능 추가


In [248]:


#Request Soup 형성
def extractSoupfromURL(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    return request, soup

#해당 URL이 유효한지 판별
def check_valid_url(soup):
    if soup.find('div', {'class': "panel-heading bg bg-info"}):
        return True
    return False

#URL이 유효했는지 로그 남김
def printLog(log, url, isURLValid):
    if isURLValid:
        print(f'{url} is valid')
        log.append(f'{url} is valid')
    else:
        print(f'{url} is invalid')
        log.append(f'{url} is invalid')

#title/step_artist/levels/bpm/category/gametype 추출
def get_title(soup):
    title = soup.find('title').prettify()
    ans = title.replace("\t", "").replace("\n", "")
    return ans[17:-19]

def get_step_artist(soup):
    step_artist = r'bg-primary">(.*?)</span>'
    step_artist_list = re.findall(step_artist, str(soup))
    return step_artist_list

def get_levels(soup):
    levels = r'Levels/(.*?).png"'
    soup_partial = soup.find('div', class_='list-group-item')
    levels_list = re.findall(levels, str(soup_partial))
    ans = level_format_list(levels_list)
    return ans
def extract_before_slash(s):
    # Split the string by '/' and take the first element
    return s.split('/')[0]

def level_format(str):
    type = extract_before_slash(str)
    level = str[-2:]
    return f'{type}{level}'

def level_format_list(levels):
    ans = []
    for i in levels:
        new_item = level_format(i)
        ans.append(new_item)
    print(ans)
    return ans

def get_bpm(soup):
    span_text = soup.find('span', {'class': 'badge bg-info'}).text
    bpm = ''.join(filter(str.isdigit, span_text))
    return bpm

def get_category(soup):
    category = soup.find('span', {'class': 'badge bg-success pull-right'}).text
    return category

def get_gametype(soup):
    gametype = soup.find('span', {'class': 'badge bg-inverse pull-right'}).text
    return gametype

#metadata 추출 - 위 function들을 하나로 합침
def extract_metadata(soup):
    title = get_title(soup)
    step_artist_list = get_step_artist(soup)
    levels_list = get_levels(soup)
    bpm = get_bpm(soup)
    category = get_category(soup)
    gametype = get_gametype(soup)
    return title, step_artist_list, levels_list, bpm, category, gametype
    

In [249]:
#metadata를 string으로 변환
def string_of_data(index, valid, Song, BPM, Category, GameType, Level, StepArtist):
    if valid == False:
        return f'index = {index} is invalid\n-\n'
    else:
        return f'index = {index}\nSong: {Song}\nBPM: {BPM}\nCategory: {Category}\nGameType: {GameType}\nLevel: {Level}\nStepArtist: {StepArtist}\n-\n'

#Utility Function: 혹시 스텝-제작자가 매칭이 되지 않을지 확인
def checkLength(list1, list2):
    if len(list1) != len(list2):
        print(f'Length of Levels: {len(list1)}\nLength of StepArtists: {len(list2)}')
    
#URL을 통해 soup을 추출하고, 유효한지 판별
def parse_URL(url_num):
    current_URL = f'https://pumpout.anyhowstep.com/songs/{url_num}'
    request, soup = extractSoupfromURL(current_URL)
    isURLValid = check_valid_url(soup)
    return current_URL, request, soup, isURLValid

#list를 string으로 변환
def get_log_string(log):
    ans = ""
    for i in log:
        ans += i + "\n"
    return ans


In [ ]:
#initialize
url_list = []
requests_list = []
soup_list = []
url_valid_list = []
log = []

for i in range(929):
    current_URL, request, soup, isURLValid = parse_URL(i+1)
    #Get URL, Request,Soup, and check if URL is valid
    url_list.append(current_URL)
    requests_list.append(request)
    soup_list.append(soup)
    isURLValid = check_valid_url(soup_list[i])
    url_valid_list.append(isURLValid)
    #printLog(log, url_list[i], isURLValid)



In [254]:
#먼저 앞으로 URL에 다시 접근하지 않아도 되도록 먼저 soup를 txt 파일로 백업했음
def BigURLString(soup_list):
    ans = ""
    for i in range(len(soup_list)):
        unitStr = f"!!URL index= {i}!!\n"
        unitStr += soup_list[i].prettify()
        unitStr += "!!End of URL!!\n\n"
        ans += unitStr
    print(i)
    return ans

strSoup = BigURLString(soup_list)

with open('Soup_backup.txt', 'w', encoding = 'utf-8') as f:
    f.write(strSoup)
    f.close()

928


In [ ]:
#metadata의 종류들을 initialize
Song_list = []
BPM_list = []
Category_list = []
Level_list = []
StepArtist_list = []
Gametype_list = []
#TODO: 출시 버전 정리

#metadata 추출
for i in range(len(soup_list)):
    if url_valid_list[i] == True:
        #if URL is valid, then start parsing.
        song_title, step_artist_list, levels_list, bpm, category, gametype = extract_metadata(soup_list[i])
        Song_list.append(song_title)
        BPM_list.append(bpm)
        Category_list.append(category)
        Level_list.append(levels_list)
        StepArtist_list.append(step_artist_list)
        Gametype_list.append(gametype)
    else:
        print(f'{url_list[i]} is invalid. Skipping...')


In [259]:
#로그 정리 (URL 파싱 성공 여부)
Writeme = ""
for i in range(915):
    Writeme += string_of_data(i, url_valid_list[i], Song_list[i], BPM_list[i], Category_list[i], Gametype_list[i], Level_list[i], StepArtist_list[i])

log_result = get_log_string(log)

In [260]:
#txt 파일로 저장
with open('output.txt', 'w') as f:
    f.write(Writeme)
    f.close()

with open('output_log.txt', 'w') as f:
    f.write(log_result)
    f.close()

In [103]:
with open('SongRanks-KR.txt', encoding = 'utf-8') as file:
    text2 = file.read()

with open('SongRanks-EN.txt', encoding = 'utf-8') as file:
    text1 = file.read()
    
def remove_tabs_and_enters(text):
    return text.replace('\t', '').replace('\n', '')

def remove_multiple_spaces(text):
    return ' '.join(text.split())

#URL 파싱 과정에서의 anomalie들 정리
def correct_anomalies_html(text):
    text = text.replace('&amp;', '&')
    text = text.replace('&quot;', '"')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&nbsp;', ' ')
    return text

#URL 파싱 과정에서의 anomalie들 정리: 특정 노래들에 대해서 정리
def correct_anomalies_custom(text):
    text = text.replace("feat.  KuTiNA", "feat. KuTiNA")
    return text

In [104]:
def extract_content_between_tags(text):
    # 정규 표현식을 사용하여 "t1">과 </p> 사이의 내용을 찾음
    pattern1 = r't1">(.*?)</p>'
    pattern2 = r't2">(.*?)</p>'
    # findall 메서드를 사용해 모든 일치 항목을 리스트로 반환
    matches1 = re.findall(pattern1, text, re.DOTALL)
    matches2 = re.findall(pattern2, text, re.DOTALL)
    return matches1, matches2
text1 = correct_anomalies_custom(correct_anomalies_html(remove_multiple_spaces(remove_tabs_and_enters(text1))))
text2 = correct_anomalies_custom(correct_anomalies_html(remove_multiple_spaces(remove_tabs_and_enters(text2))))
songname_EN, composer_EN = extract_content_between_tags(text1)
songname_KR, composer_KR = extract_content_between_tags(text2)

with open('MetaData-ComposerSongNames.txt', 'w', encoding = 'utf-8') as file:
    for i in range(544):
        file.write(f'{songname_KR[i]}//{composer_KR[i]}//{songname_EN[i]}//{composer_EN[i]}\n')


In [ ]:

#혹시 double space가 더 남아있는기 검증했었음
def detect_double_spaces(text):
    return '  ' in text
'''
with open('MetaData-ComposerSongNames.txt', encoding = 'utf-8') as file:
    text_metadata = file.read()

print(detect_double_spaces(text_metadata))
'''